In [35]:
#importing libraries
import tensorflow as tf
import sklearn.model_selection as skms
import sklearn.preprocessing as skp
from tensorflow import keras
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import pandas as pd
import numpy as np
import random
seed = 12
np.random.seed(seed)
from sklearn import model_selection as skms
import numpy as np
import librosa
import warnings
import glob
from tkinter import *
from tkinter import filedialog
from tkinter import messagebox
from path import Path
import imageio
import pandas as pd
from keras.models import model_from_json
import pickle

In [36]:
#loading models
json_file = open('model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
loaded_model.load_weights("model.h5")


json_file2 = open('model2.json', 'r')
loaded_model_json2 = json_file2.read()
json_file2.close()
loaded_model2 = model_from_json(loaded_model_json2)
loaded_model2.load_weights("model2.h5")

json_file3 = open('model3.json', 'r')
loaded_model_json3 = json_file3.read()
json_file3.close()
loaded_model3 = model_from_json(loaded_model_json3)
loaded_model3.load_weights("model3.h5")

loaded_model4 = pickle.load(open("model4.pkl", "rb"))

loaded_model5 = pickle.load(open("model5.pkl", "rb"))

In [40]:
#code

df = pd.read_csv('features_3_sec.csv')
label_index = dict()
index_label = dict()
for i, x in enumerate(df.label.unique()):
    label_index[x] = i
    index_label[i] = x
df.label = [label_index[l] for l in df.label]
df_shuffle = df.sample(frac=1, random_state=11).reset_index(drop=True)
df_X=df_shuffle.iloc[:,2:-1]
df_y=df_shuffle.iloc[:,-1]
X_train, X_test, y_train, y_test = skms.train_test_split(df_X, df_y, train_size=0.9, random_state=seed, stratify=df_y)
from sklearn import preprocessing as skp
scaler = skp.StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test =scaler.transform(X_test)

def audio_genre():
    
    warnings.filterwarnings('ignore')
    path=direc.get()
    try:
        x, sr = librosa.load(path)
    except:
        messagebox.showinfo("Error occured!","Please enter a valid path")
    c_s=librosa.feature.chroma_stft(y=x, sr=sr)
    c_s_m=np.mean(c_s)
    c_s_v=np.var(c_s)
    rms=librosa.feature.rms(y=x)
    rms_m=np.mean(rms)
    rms_var=np.var(rms)
    s_c=librosa.feature.spectral_centroid(y=x,sr=sr)
    s_c_m=np.mean(s_c)
    s_c_v=np.var(s_c)
    s_b = librosa.feature.spectral_bandwidth(y=x, sr=sr)
    s_b_m=np.mean(s_b)
    s_b_v=np.var(s_b)
    r=librosa.feature.spectral_rolloff(y=x, sr=sr)
    r_m=np.mean(r)
    r_v=np.var(r)
    zcr=librosa.feature.zero_crossing_rate(y=x)
    zcr_m=np.mean(zcr)
    zcr_v=np.var(zcr)
    y_h = librosa.effects.harmonic(y=x)
    y_h_m=np.mean(y_h)
    y_h_v=np.var(y_h)
    per=librosa.effects.percussive(y=x) 
    per_m=np.mean(per)
    per_v=np.var(per)
    te=librosa.beat.tempo(x)
    dict1={'c_s_m':[c_s_m],'c_s_v':[c_s_v],'rms_m':[rms_m],'rms_var':[rms_var],'s_c_m':[s_c_m],'s_c_v':[s_c_v],'s_b_m':[s_b_m],'s_b_v':[s_b_v],'r_m':[r_m],'r_v':[r_v],'zcr_m':[zcr_m],'zcr_v':[zcr_v],'y_h_m':[y_h_m],'y_h_v':[y_h_v],'per_m':[per_m],'per_v':[per_v],'t':[te[0]]}
    mfc=librosa.feature.mfcc(y=x, sr=sr)
    mfc1_m=np.mean(mfc[0])
    for i in range(1,21):
        mfcm=np.mean(mfc[i-1])
        mfcv=np.var(mfc[i-1])
        st='mfcc_'+str(i)+'_m'
        dict1[st]=[mfcm]
        st1='mfcc_'+str(i)+'_v'
        dict1[st1]=[mfcv]
    d_predict = pd.DataFrame(dict1)
    d_predict_s =scaler.transform(d_predict)
    d_predict_1=loaded_model.predict(d_predict_s)
    d_predict_2=loaded_model2.predict(d_predict_s)
    d_predict_3=loaded_model3.predict(d_predict_s)
    d_predict_4=loaded_model4.predict(d_predict_s)
    d_predict_5=loaded_model5.predict(d_predict_s)
    d1=[np.argmax(i) for i in d_predict_1]
    d2=[np.argmax(i) for i in d_predict_2]
    d3=[np.argmax(i) for i in d_predict_3]
    d4=[np.argmax(i) for i in d_predict_4]
    d5=[d_predict_5[0]]

    freq=[0,0,0,0,0,0,0,0,0,0]
    yo=d1[0]
    freq[yo]=freq[yo]+1
    yo2=d2[0]
    freq[yo2]=freq[yo2]+1
    yo3=d3[0]
    freq[yo3]=freq[yo3]+1
    yo4=d4[0]
    freq[yo4]=freq[yo4]+1
    yo5=d5[0]
    freq[yo5]=freq[yo5]+1
    ma=np.max(freq)
    yt=freq.index(ma)
    c=0
    d=0
    for j in freq:
        if j==1:
            c=c+1
        if j==2:
            d=d+1
    if c==5:
        yt=d3[0]
    if d==2:
        yt=d3[0]
        if freq[yo3]==1:
            yt=d2[0]
    st="The music genre is "+index_label[yt]
    messagebox.showinfo("Music Genre Classifier",st)


In [43]:
#interface

wn=Tk()
wn.title("Music Genre classifier")
wn.geometry('500x300')
wn.config(bg='azure')

extension=StringVar(wn)
direc=StringVar(wn)
Label(wn,text='Music Genre Classifier',bg='azure',fg='black',font=('Times',20,'bold')).place(x=60,y=10)
Label(wn,text='Please enter the path of the song on your pc',bg='azure2',anchor="e",justify=LEFT).place(x=20,y=130)
Entry(wn,textvariable=direc,width=35,font=('calibore',10,'normal')).place(x=200,y=160)
Label(wn,text='Please click the button to get the Muisc Genre',bg='azure2',anchor="e",justify=LEFT).place(x=20,y=190)
Button(wn,text="Click Me",bg='ivory3',font=('calibore',13),command=audio_genre).place(x=230,y=220)
wn.mainloop()